In [ ]:
# ================================
# Prediciton
# ================================

"""
This notebook:
- Lets you upload a .wav file
- Predicts emotion using XGBoost or CNN


Models must be pre-trained and saved:
- cnn_model.h5 or xgb_model.joblib
- label_encoder.pkl
"""

# 📦 Imports
import librosa
import numpy as np
import joblib
import tensorflow as tf
from google.colab import files
import openai

# ✅ Load files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# ✅ Paths
XGB_MODEL_PATH = "/content/drive/MyDrive/capstone_data/xgb_model.joblib"
CNN_MODEL_PATH = "/content/drive/MyDrive/capstone_data/cnn_model.h5"
ENCODER_PATH   = "/content/drive/MyDrive/capstone_data/label_encoder.pkl"

# ✅ Load models
xgb_model = joblib.load(XGB_MODEL_PATH)
cnn_model = tf.keras.models.load_model(CNN_MODEL_PATH)
label_encoder = joblib.load(ENCODER_PATH)

# ✅ Feature extraction for XGBoost
def extract_features_flat(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant') if pad_width > 0 else mfccs[:, :max_pad_len]
        return mfccs.flatten().reshape(1, -1)
    except Exception as e:
        print(f"[ERROR] XGB feature extraction failed: {e}")
        return None

# ✅ Feature extraction for CNN
def extract_features_2d(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant') if pad_width > 0 else mfccs[:, :max_pad_len]
        return mfccs[..., np.newaxis].reshape(1, 40, max_pad_len, 1)
    except Exception as e:
        print(f"[ERROR] CNN feature extraction failed: {e}")
        return None

# ✅ Predict emotion with both models
flat_feat = extract_features_flat(file_path)
cnn_feat  = extract_features_2d(file_path)

if flat_feat is not None:
    xgb_pred = xgb_model.predict(flat_feat)[0]
    xgb_emotion = label_encoder.inverse_transform([xgb_pred])[0]
    print(f"🎯 XGBoost Predicted Emotion: {xgb_emotion}")

if cnn_feat is not None:
    cnn_pred = np.argmax(cnn_model.predict(cnn_feat), axis=1)[0]
    cnn_emotion = label_encoder.inverse_transform([cnn_pred])[0]
    print(f"🤖 CNN Predicted Emotion: {cnn_emotion}")
